
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

In [6]:
sales_rep_agent = Agent(
    role="Medical Sales Specialist",
    goal="Identify healthcare institutions that could benefit from our medical solutions",
    backstory=(
        "As a member of the medical sales team, your mission is to analyze data and trends "
        "in the healthcare sector to find potential clients for our innovative medical products. "
        "Your work is crucial in connecting healthcare providers with solutions that can improve patient outcomes."
    ),
    allow_delegation=False,
    verbose=True
)

In [7]:
lead_sales_rep_agent = Agent(
    role="Medical Outreach Coordinator",
    goal="Engage with healthcare decision-makers through personalized communications",
    backstory=(
        "In the medical outreach team, you are the key communicator who bridges the gap "
        "between our medical solutions and the healthcare providers who can implement them. "
        "By crafting personalized messages, you help healthcare leaders understand how our products "
        "can enhance their services and benefit their patients."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [8]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [9]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [10]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.

- When running locally, you can customize the code with your logic in the `_run` function.

In [11]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [12]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [14]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, a healthcare institution in the {industry} sector "
        "that has potential interest in our medical solutions. Utilize all available data sources to compile "
        "a detailed profile, focusing on key decision-makers, recent developments in their medical services "
        "or research, and potential needs that align with our offerings. This task is crucial for tailoring "
        "our engagement strategy effectively.\nDon't make assumptions and only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, including institutional background, key medical personnel, "
        "recent medical achievements or expansions, and identified needs. Highlight potential areas where our "
        "medical solutions can provide value, and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [15]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from the profiling report on {lead_name}, craft a personalized outreach "
        "campaign aimed at {key_decision_maker}, the {position} of {lead_name}. The campaign should address "
        "their recent {milestone} in healthcare or research and how our medical solutions can support their goals. "
        "Your communication must resonate with {lead_name}'s mission and values in patient care, demonstrating "
        "a deep understanding of their medical focus and needs.\nDon't make assumptions and only use information "
        "you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized communication drafts tailored to {lead_name}, specifically targeting "
        "{key_decision_maker}. Each draft should include a compelling narrative that connects our medical "
        "solutions with their recent achievements in healthcare and future objectives. Ensure the tone is "
        "engaging, professional, and aligned with {lead_name}'s commitment to patient care and medical excellence."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [16]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew



In [17]:

inputs = {
    "lead_name": "Chagua Health",
    "industry": "Healthcare",
    "key_decision_maker": "Dr. Koech",
    "position": "CEO",
    "milestone": "opening of a new cardiac care center"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Medical Sales Specialist
 [INFO]: == Starting Task: Conduct an in-depth analysis of Chagua Health, a healthcare institution in the Healthcare sector that has potential interest in our medical solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent developments in their medical services or research, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about Chagua Health to create a comprehensive report for potential engagement strategies.

Action: Read a file's content
Action Input: {"file_path": "./instructions/Chagua_Health_Profile.txt"} 

I encountered an error while trying to use the tool. This was the error: [Errno 2] No such file or directory: './instructions

Thought: I need to gather information about Chagua Health using the available tools.
Action: Search the internet
Action Input: {"search_query": "Chagua Health Overview"} 


Search results: Title: Chagua Health Overview | SignalHire Company Profile
Link: https://www.signalhire.com/companies/chagua-health
Snippet: At Chagua Health, we are dedicated to revolutionizing healthcare by leveraging innovative technologies to provide better access, affordability, ...
---
Title: Chagua PrEP, Chagua Life Campaign from Kiambu County, Kenya
Link: https://www.prepwatch.org/resources/chagua-prep-chagua-life-campaign-from-kiambu-county-kenya/
Snippet: This tool was developed to be used by peer outreach workers, peer educators, and other community-based health providers to help provide ...
---
Title: Deborah koech - Founder - Chagua Health | LinkedIn
Link: https://ke.linkedin.com/in/deborah-koech-98a238223
Snippet: Deborah Koech is a health tech enthusiast passionate about integrating healthcare and tec

I need to ensure that my personalized outreach campaign to Dr. Koech is engaging, professional, and aligned with Chagua Health's mission and values. I must demonstrate a deep understanding of their recent achievements in healthcare, specifically the opening of a new cardiac care center, and show how our medical solutions can support their goals effectively.

Action: Sentiment Analysis Tool
Action Input: {"text": "Dear Dr. Koech, I am reaching out to congratulate you on the successful opening of the new cardiac care center at Chagua Health. Your dedication to revolutionizing healthcare through innovative technologies is truly inspiring. As a key figure at Chagua Health, your focus on integrating healthcare and technology for innovation is commendable. I believe that our medical solutions can complement your efforts and help further enhance the quality of care provided at Chagua Health. I look forward to the opportunity to discuss how we can collaborate to support your goals in patient c

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Draft 1:
Subject: Congratulations on the Opening of the New Cardiac Care Center

Dear Dr. Koech,

I am reaching out to extend my heartfelt congratulations on the successful opening of the new cardiac care center at Chagua Health. Your dedication to revolutionizing healthcare through innovative technologies is truly inspiring. As a key figure at Chagua Health, your focus on integrating healthcare and technology for innovation is commendable. 

I believe that our medical solutions can complement your efforts and help further enhance the quality of care provided at Chagua Health. Our cutting-edge technology and commitment to patient care align seamlessly with Chagua Health's mission and values. I am confident that a collaboration between our teams can lead to significant advancements in patient outcomes and medical excellence.

I look forward to the opportunity to discuss how we can work together to support your goals and continue delivering exceptional care to the community.

Warm regards,
[Your Name]
Medical Outreach Coordinator

Draft 2:
Subject: Enhancing Patient Care through Collaboration

Dear Dr. Koech,

I wanted to take a moment to acknowledge the recent milestone achieved by Chagua Health with the opening of the new cardiac care center. Your relentless pursuit of integrating healthcare and technology for innovation is truly commendable and aligns perfectly with our mission to improve patient outcomes through advanced medical solutions.

At [Your Company Name], we are dedicated to supporting healthcare providers like Chagua Health in delivering high-quality care to their patients. Our innovative products have been designed to enhance the efficiency and effectiveness of medical services, ultimately leading to better outcomes for patients.

I am excited about the possibility of collaborating with Chagua Health to further elevate the standard of care provided to the community. Together, we can make a meaningful impact on the lives of those we serve.

I look forward to the opportunity to discuss how we can work together to achieve our shared goals.

Warm regards,
[Your Name]
Medical Outreach Coordinator